## 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
df_new = pd.read_csv('/content/drive/MyDrive/Streamlit 해커톤/data/df_new.csv')
df_new

## GPT-3.5-turbo API를 이용하여 상품과 질문 매핑

In [ ]:
!pip install openai

### 7개의 카테고리를 대표하는 값 매핑

In [ ]:
labels=[]

In [ ]:
import os
import openai

openai.api_key = ""

def labeling_gpt(title, body, category):
    messages = [
     {"role": "system", "content":
       "내가 음식 제품을 제시해주면, 너는 카테고리별로 대표값을 정해야 돼.\
        각 카테고리별 선택지에서 제품의 특징과 가장 유사한 선택지를 하나가 대표값이 되는거야. 카테고리별 선택지는 다음과 같아. 선택지 중 단 하나만 선택해야 해.\
        1번 카테고리: {'즉석식품-간편', '요리재료-소스'}, \
        2번 카테고리: {'밥', '빵', '면', '떡', '튀김'}, \
        3번 카테고리: {'한식', '일식', '중식', '양식'}, \
        4번 카테고리: {'매움-매콤-고추-와사비', '맵지 않음'}, \
        5번 카테고리: {'달다-달콤,-당', '달지 않음'}, \
        6번 카테고리: {'육식', '채식-과일'}, \
        7번 카테고리: {'새콤-신맛-시다', '시지 않음'}\
        총 7개의 카테고리니까 총 7개의 대표값을 정해야 해. 각 카테고리의 대표값만 출력해야 해."},

    {"role": "user", "content": " \
      다음 제품의 카테고리별 대표값을 출력해줘. 제품 이름 : " + '"' + title + '"' + "제품 설명 : " + '"' + body + '"' + "제품 분류 : " + '"' + category + '"'},

    {"role": "assistant", "content" : "제품의 특징과 카테고리별 선택지의 유사도를 잘 구해서 대표값을 선택해야 해.예를 들어, \
        '라면'이 입력되면 다음과 같은 양식으로 출력해야 해. \
        '1번 카테고리: 즉석식품-간편, 2번 카테고리: 면. 3번 카테고리: 한식, 4번 카테고리: 매움-매콤-고추-와사비, 5번 카테고리: 달지 않음, 6번 카테고리: 채식-과일, 7번 카테고리: 시지 않음'"}
    ]

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.2,
        top_p=0
    )

    print(completion.choices[0].message.content)
    labels.append(completion.choices[0].message.content)


In [ ]:
for i in range(0, 33):
  title = df_new['title'][i]
  body = df_new['body'][i]
  category = df_new['category'][i]
  print(title)
  labeling_gpt(title, body, category)

In [ ]:
len(labels)

### 텍스트 전처리 용도로 다시 gpt api 사용

In [ ]:
labels_2 = []

In [ ]:
def labeling_gpt2(data):
    messages = [
     {"role": "system", "content":
       "필요한 값은 '카테고리명과 값'만 이야.문장을 제외하고 필요한 값만 남겨줘"},

    {"role": "user", "content": " \
      다음 " + data + "에서 '카테고리명과 값'만 남기는거야." },

    {"role": "assistant", "content" : "예를 들어 다음과 같은 양식으로 출력해야 해. \
        '1번 카테고리: 즉석식품-간편, 2번 카테고리: 면. 3번 카테고리: 한식, 4번 카테고리: 매움-매콤-고추-와사비, 5번 카테고리: 달지 않음, 6번 카테고리: 채식-과일, 7번 카테고리: 시지 않음' \
        만약 이미 '카테고리명과 값'만 존재한다면 그대로 두면 돼."}
    ]

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.2,
        top_p=0
    )

    print(completion.choices[0].message.content)
    labels_2.append(completion.choices[0].message.content)

In [ ]:
for i in labels:
  labeling_gpt2(i)

### 'category' 칼럼 추가

In [ ]:
df_new['category'] = labels_2

In [ ]:
df_new

In [ ]:
pd.set_option('display.max_colwidth', 1000)

In [ ]:
# 이상한 값 변경
df_new.loc[df_new['title']  == '꼬리까지 가득 찬 팥붕어빵 480G', 'category'] = "1번 카테고리: 즉석식품-간편\n2번 카테고리: 빵\n3번 카테고리: 한식\n4번 카테고리: 맵지 않음\n5번 카테고리: 달다-달콤,-당\n6번 카테고리: 육식\n7번 카테고리: 시지 않음"
df_new.loc[df_new['title']  == '베이컨크림뇨끼', 'category'] = "1번 카테고리: 즉석식품-간편\n2번 카테고리: 면\n3번 카테고리: 양식\n4번 카테고리: 맵지 않음\n5번 카테고리: 달다-달콤,-당\n6번 카테고리: 육식\n7번 카테고리: 시지 않음"
df_new.loc[df_new['title']  == '참깨라면 볶음면', 'category'] = "1번 카테고리: 즉석식품-간편\n2번 카테고리: 면\n3번 카테고리: 한식\n4번 카테고리: 매움-매콤-고추-와사비\n5번 카테고리: 달지 않음\n6번 카테고리: 채식-과일\n7번 카테고리: 시지 않음"
df_new.loc[df_new['title']  == '트러플크림뇨끼', 'category'] = "1번 카테고리: 요리재료-소스\n2번 카테고리: 면\n3번 카테고리: 양식\n4번 카테고리: 맵지 않음\n5번 카테고리: 달다-달콤,-당\n6번 카테고리: 육식\n7번 카테고리: 시지 않음"
df_new.loc[df_new['title']  == '청송사과로만든  순사과식초', 'category'] = "1번 카테고리: 요리재료-소스\n2번 카테고리: 빵\n3번 카테고리: 한식\n4번 카테고리: 맵지 않음\n5번 카테고리: 달지 않음\n6번 카테고리: 채식-과일\n7번 카테고리: 새콤-신맛-시다"
df_new.loc[df_new['title']  == '황도 통조림', 'category'] = "1번 카테고리: 즉석식품-간편\n2번 카테고리: 빵\n3번 카테고리: 양식\n4번 카테고리: 맵지 않음\n5번 카테고리: 달다-달콤,-당\n6번 카테고리: 채식-과일\n7번 카테고리: 시지 않음"
df_new.loc[df_new['title']  == '이천쌀로만든  막걸리식초', 'category'] = "1번 카테고리: 요리재료-소스\n2번 카테고리: 빵\n3번 카테고리: 한식\n4번 카테고리: 맵지 않음\n5번 카테고리: 달지 않음\n6번 카테고리: 채식-과일\n7번 카테고리: 새콤-신맛-시다"
df_new.loc[df_new['title']  == '오뚜기밥 수향미', 'category'] = "1번 카테고리: 즉석식품-간편\n2번 카테고리: 밥\n3번 카테고리: 한식\n4번 카테고리: 맵지 않음\n5번 카테고리: 달지 않음\n6번 카테고리: 채식-과일\n7번 카테고리: 시지 않음"
df_new.loc[df_new['title']  == '마열라면', 'category'] = "1번 카테고리: 즉석식품-간편\n2번 카테고리: 면\n3번 카테고리: 한식\n4번 카테고리: 매움-매콤-고추-와사비\n5번 카테고리: 달지 않음\n6번 카테고리: 채식-과일\n7번 카테고리: 시지 않음"
df_new.loc[df_new['title']  == '튀만전찍먹소스 320G', 'category'] = "1번 카테고리: 요리재료-소스\n2번 카테고리: 튀김\n3번 카테고리: 한식\n4번 카테고리: 매움-매콤-고추-와사비\n5번 카테고리: 달지 않음\n6번 카테고리: 채식-과일\n7번 카테고리: 새콤-신맛-시다"
df_new.loc[df_new['title']  == '빅컵누들 매콤한맛', 'category'] = "1번 카테고리: 즉석식품-간편\n2번 카테고리: 면\n3번 카테고리: 한식\n4번 카테고리: 매움-매콤-고추-와사비\n5번 카테고리: 달지 않음\n6번 카테고리: 채식-당\n7번 카테고리: 시지 않음"
df_new.loc[df_new['title']  == '직접 갈아먹는 스모크페퍼솔트 70g', 'category'] = "1번 카테고리: 즉석식품-간편\n2번 카테고리: 면\n3번 카테고리: 양식\n4번 카테고리: 맵지 않음\n5번 카테고리: 달지 않음\n6번 카테고리: 채식-과일\n7번 카테고리: 시지 않음"
df_new.loc[df_new['title']  == '순후추떡볶이', 'category'] = "1번 카테고리: 즉석식품-간편\n2번 카테고리: 떡\n3번 카테고리: 한식\n4번 카테고리: 매움-매콤-고추-와사비\n5번 카테고리: 달지 않음\n6번 카테고리: 채식-과일\n7번 카테고리: 시지 않음"
df_new.loc[df_new['title']  == '꼬리까지 가득 찬 슈크림붕어빵 480G', 'category'] = "1번 카테고리: 즉석식품-간편\n2번 카테고리: 빵\n3번 카테고리: 한식\n4번 카테고리: 맵지 않음\n5번 카테고리: 달다-달콤,-당\n6번 카테고리: 채식-과일\n7번 카테고리: 시지 않음"
df_new.loc[df_new['title']  == '키위 저칼로리드레싱(냉장) 310G', 'category'] = "1번 카테고리: 요리재료-소스\n2번 카테고리: 빵\n3번 카테고리: 양식\n4번 카테고리: 맵지 않음\n5번 카테고리: 달다-달콤,-당\n6번 카테고리: 채식-과일\n7번 카테고리: 새콤-신맛-시다"
df_new.loc[df_new['title']  == '제주담음 제주감귤도우피자', 'category'] = "1번 카테고리: 즉석식품-간편\n2번 카테고리: 빵\n3번 카테고리: 양식\n4번 카테고리: 맵지 않음\n5번 카테고리: 달다-달콤,-당\n6번 카테고리: 채식-과일\n7번 카테고리: 새콤-신맛-시다"
df_new.loc[df_new['title']  == '직접 볶은 트러플 간짜장 고형', 'category'] = "1번 카테고리: 요리재료-소스\n2번 카테고리: 면\n3번 카테고리: 중식\n4번 카테고리: 맵지 않음\n5번 카테고리: 달다-달콤,-당\n6번 카테고리: 채식-과일\n7번 카테고리: 시지 않음"
df_new.loc[df_new['title']  == '히모카와우동(2인) 396G', 'category'] = "1번 카테고리: 요리재료-소스\n2번 카테고리: 면\n3번 카테고리: 일식\n4번 카테고리: 맵지 않음\n5번 카테고리: 달지 않음\n6번 카테고리: 채식-과일\n7번 카테고리: 시지 않음"
df_new.loc[df_new['title']  == '빅컵누들 우동맛', 'category'] = "1번 카테고리: 즉석식품-간편\n2번 카테고리: 면\n3번 카테고리: 일식\n4번 카테고리: 맵지 않음\n5번 카테고리: 달지 않음\n6번 카테고리: 채식-과일\n7번 카테고리: 시지 않음"

In [ ]:
 df_new

## 하나의 카테고리 변수에 들어있던 7개의 카테고리를 분류하여 각각의 칼럼으로 나눔

In [ ]:
df_new['readymeal'] = df_new['category'].str.extract(r'1번 카테고리: ([^\n]+)')

In [ ]:
df_new['kind'] = df_new['category'].str.extract(r'2번 카테고리: ([^\n]+)')

In [ ]:
df_new['country'] = df_new['category'].str.extract(r'3번 카테고리: ([^\n]+)')

In [ ]:
df_new['spicy'] = df_new['category'].str.extract(r'4번 카테고리: ([^\n]+)')

In [ ]:
df_new['sweet'] = df_new['category'].str.extract(r'5번 카테고리: ([^\n]+)')

In [ ]:
df_new['began'] = df_new['category'].str.extract(r'6번 카테고리: ([^\n]+)')

In [ ]:
df_new['sour'] = df_new['category'].str.extract(r'7번 카테고리: ([^\n]+)')

In [ ]:
df_new[['readymeal','kind','country','spicy','sweet','began','sour']]

In [ ]:
# 이상한 값 바꾸기
df_new['title'][8]

In [ ]:
df_new['readymeal'][8] = '즉석식품-간편'

In [ ]:
df_new['sweet'][8] = '달다-달콤,-당'

In [ ]:
df_new['began'][8] = '육식'

In [ ]:
df_new['sour'][8] = '새콤-신맛-시다'

In [ ]:
df_new['began'][12] = '채식-과일'

In [ ]:
df_new[['readymeal','kind','country','spicy','sweet','began','sour']]

### 원핫 인코딩

In [ ]:
# 원핫 인코딩 적용
df_encoded = pd.get_dummies(df_new, columns=['readymeal', 'kind', 'country', 'spicy', 'sweet', 'began', 'sour'])

In [ ]:
df_encoded

In [ ]:
df_new_encoded = df_encoded.drop(columns=[''])

In [ ]:
# 칼럼명 변경
new_column_names = {'readymeal_요리재료-소스': 'readymeal_요리재료',
                    'readymeal_즉석식품-간편': 'readymeal_즉석식품',
                    'spicy_매움-매콤-고추-와사비': 'spicy_매움',
                    'spicy_맵지 않음': 'spicy_맵지않음',
                    'sweet_달다-달콤,-당': 'sweet_달다',
                    'sweet_달지 않음':'sweet_달지않음',
                    'began_채식-과일':'began_채식',
                    'sour_새콤-신맛-시다':'sour_신맛',
                    'sour_시지 않음':'sour_시지않음'}

In [ ]:
df_new_encoded.rename(columns=new_column_names, inplace=True)

In [ ]:
df_new_encoded.to_csv("df_new_encoded.csv", encoding = "utf-8-sig")